In [72]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from itertools import product
from scipy.stats import qmc  # For Latin Hypercube Sampling
import torch
import gpytorch

from PIL import Image
from datetime import datetime


In [73]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, meanPrior):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        if meanPrior == 'max':
            # self.mean_module = gpytorch.means.ZeroMean()
            self.mean_module = gpytorch.means.ConstantMean()
            # self.mean_module.constant = torch.nn.Parameter(torch.tensor(torch.max(train_y)))
            self.mean_module.constant.data = torch.tensor(torch.max(train_y))

        else:
            # self.mean_module = gpytorch.means.ConstantMean(constant_prior=torch.max(train_y))
            self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
    
def GPTrain(features, targets, meanPrior):

    tensorSamplesXY = torch.from_numpy(features)
    tensorSamplesZ = torch.from_numpy(targets)

    likelihood = gpytorch.likelihoods.GaussianLikelihood() 
    model = ExactGPModel(tensorSamplesXY, tensorSamplesZ, likelihood, meanPrior)
    likelihood.noise = 1e-4
    likelihood.noise_covar.raw_noise.requires_grad_(False)

    training_iter = 250
    # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.05)  # Includes GaussianLikelihood parameters

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    for i in range(training_iter):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(tensorSamplesXY)
        # Calc loss and backprop gradients
        loss = -mll(output, tensorSamplesZ)
        loss.backward()
        # print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        #     i + 1, training_iter, loss.item(),
        #     model.covar_module.base_kernel.lengthscale.item(), #.kernels[0] after base_kernel if have multiple kernels
        #     model.likelihood.noise.item()
        # ))
        optimizer.step()
    
    return model


def GPEval(model, newFeatures):
    model.eval()

    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        observed_pred = model(torch.from_numpy(newFeatures))

    mean_pred = observed_pred.mean.numpy()

    return mean_pred

In [74]:
class DifferentialEvolution:
    def __init__(self, bounds, objective_function, pop_size=50, mutation_factor=0.8, crossover_prob=0.7, max_generations=200, method='random'):
        """
        Initialize the Differential Evolution (DE) optimizer.
        
        Parameters:
        bounds (list of tuple): List of (min, max) bounds for each dimension.
        pop_size (int): Number of candidate solutions in the population.
        mutation_factor (float): Scaling factor for mutation [0, 2].
        crossover_prob (float): Crossover probability [0, 1].
        max_generations (int): Maximum number of generations to evolve.
        method (str): Population initialization method ('random' or 'lhs').
        """
        self.bounds = np.array(bounds)
        self.dimensions = len(bounds)
        self.pop_size = pop_size
        self.mutation_factor = mutation_factor
        self.crossover_prob = crossover_prob
        self.max_generations = max_generations
        self.method = method
        
        # Initialize population
        self.population = self.initialize_population()
        self.best_solution = None
        self.best_fitness = np.inf
        self.objective_function = objective_function
    
    def initialize_population(self):
        """Initialize population using random sampling or Latin Hypercube Sampling."""
        if self.method == 'lhs':
            # Latin Hypercube Sampling
            sampler = qmc.LatinHypercube(d=self.dimensions)
            sample = sampler.random(n=self.pop_size)
            population = qmc.scale(sample, self.bounds[:, 0], self.bounds[:, 1])
        else:
            # Random Sampling
            population = np.random.rand(self.pop_size, self.dimensions)
            for i in range(self.dimensions):
                population[:, i] = self.bounds[i, 0] + population[:, i] * (self.bounds[i, 1] - self.bounds[i, 0])
        
        # print(population.shape)
        return population
    
    def mutate(self, target_idx):
        """Mutation using DE/best/1 strategy."""
        # Choose three random and distinct individuals different from target_idx
        indices = [idx for idx in range(self.pop_size) if idx != target_idx]
        np.random.shuffle(indices)
        r1, r2 , r3= indices[:3]
        
        # Best individual in current population

        # print(self.population.shape)

        #TODO  instead of this list comprehension bollocks just evaluate them all at once
        #as thats what i think it wants, then find the minimum of the results. 


        predictedValues = GPEval(self.objective_function, self.population)

        best_idx = np.argsort(predictedValues)[:1]

        best = self.population[best_idx]

        # best_idx = np.argmin([self.objective_function.predict(ind) for ind in self.population])
        # best = self.population[best_idx]
        
        # Mutant vector: v = best + F * (r1 - r2)
        mutant = best + self.mutation_factor * (self.population[r1] - self.population[r2])
        
        # Ensure mutant vector is within bounds
        mutant = np.clip(mutant, self.bounds[:, 0], self.bounds[:, 1])
        
        return mutant
    
    def crossover(self, target, mutant):
        """Crossover to create trial vector."""
        trial = np.copy(target)
        # print(trial.shape)
        # print(mutant.shape)
        for i in range(self.dimensions):
            if np.random.rand() < self.crossover_prob or i == np.random.randint(self.dimensions):
                # print(trial[i], mutant[i])
                trial[i] = mutant[i]
        return trial
    
    def select(self, target, trial):
        """Selection: Return the individual with the better fitness."""
        if self.objective_function.predict(trial) < self.objective_function.predict(target):
            return trial
        return target

    def select(self, target, trial):
        """Selection: Return the individual with the better fitness."""
        if GPEval(self.objective_function, trial) < GPEval(self.objective_function, target):
            return trial
        return target
    
    def optimize(self):
        """Run the Differential Evolution optimization."""
        # x_range = np.linspace(-5, 5, 100)
        # y_range = np.linspace(-5, 5, 100)
        # X, Y = np.meshgrid(x_range, y_range)
        # Z = ackley_function(X, Y)
        x_range = np.linspace(self.bounds[0,0], self.bounds[0,1],50)
        y_range = np.linspace(self.bounds[1,0], self.bounds[1,1],50)
        fullRange = list(product(x_range, y_range))
        fullRangeArray = np.array(fullRange)
        # y_pred = self.objective_function.predict(fullRangeArray)
        y_pred = GPEval(self.objective_function, fullRangeArray)

        for generation in range(self.max_generations):
            new_population = np.zeros_like(self.population)
            allTrials = np.zeros_like(self.population)
            allTargets = np.zeros_like(self.population)
            # print(self.population.shape)
            for i in range(self.pop_size):
                target = self.population[i]
                # print('break')
                # print(i)
                mutant = self.mutate(i)
                # print(mutant)
                mutant = np.reshape(mutant, (2,))
                # print(mutant)
                trial = self.crossover(target, mutant)
                trial = np.reshape(trial, (1,-1))
                target = np.reshape(target, (1,-1))
                # print('for select', trial.shape, target.shape)
                new_population[i] = self.select(target, trial)
            
            # Update the population
            self.population = new_population
            
            # Track the best solution
            # best_idx = np.argmin([self.objective_function.predict(ind) for ind in self.population])
            # best_fitness = self.objective_function.predict(self.population[best_idx])
            
            # predictedValues = self.objective_function.predict(self.population)
            predictedValues = GPEval(self.objective_function, self.population)

            best_idx = np.argsort(predictedValues)[:1]

            # best_fitness = self.objective_function.predict(self.population[best_idx])
            best_fitness = GPEval(self.objective_function, self.population[best_idx])

            if best_fitness < self.best_fitness:
                self.best_fitness = best_fitness
                self.best_solution = self.population[best_idx]
            
            # plt.contourf(x_range, y_range, y_pred, levels=50, cmap='viridis')
        plt.scatter(fullRangeArray[:,0], fullRangeArray[:,1], c = y_pred)

        plt.scatter(self.population[:, 0], self.population[:, 1], color='red', label='Final Population', s=5)
        # plt.scatter(best_solution[0], best_solution[1], color='blue', label='Best Solution', s=100)
        # plt.legend()
        plt.title("Local Surrogate")
        plt.colorbar()
        plt.clim(0,14)
        plt.xlim(self.bounds[0,0], self.bounds[0,1])
        plt.ylim(self.bounds[1,0], self.bounds[1,1])
        plt.savefig('localGP.png')
        plt.close()
        # # Debug information
        print(f"Generation {generation + 1}: Best RBF Fitness = {self.best_fitness}")
        
        return self.best_solution, self.best_fitness


In [75]:
def ackley_function(x, y, a=20, b=0.2, c=2 * np.pi):
    term1 = -a * np.exp(-b * np.sqrt(0.5 * (x**2 + y**2)))
    term2 = -np.exp(0.5 * (np.cos(c * x) + np.cos(c * y)))
    return term1 + term2 + a + np.exp(1)

# def ackley_function(x, y, a=20, b=0.2, c=2 * np.pi):

#     z = np.sin(x)+(x*np.cos(0.5*y))
#     return z

def lipschitz_global_underestimate(f_values, samplesXY, L, test_points):
    """
    Compute the Lipschitz-based global underestimate at specific points.

    Parameters:
    f_values: np.ndarray of shape (n_samples,)
        The function values at the sampled points.
    samplesXY: np.ndarray of shape (n_samples, 2)
        Array of the sampled (x, y) points.
    L: float
        The Lipschitz constant.
    test_points: np.ndarray of shape (n, 2)
        Array of (x, y) points at which to compute the underestimate.

    Returns:
    Z_under: np.ndarray of shape (n,)
        The global Lipschitz-based underestimate values at the test points.
    """
    n_test_points = test_points.shape[0]
    Z_under = np.full(n_test_points, -np.inf)  # Initialize with very low values

    # Loop over all sample points to compute their individual underestimates
    for i, (x_i, y_i) in enumerate(samplesXY):
        f_x_i_y_i = f_values[i]
        
        # Compute the distance from each test point to the sample point (x_i, y_i)
        distances = np.sqrt((test_points[:, 0] - x_i)**2 + (test_points[:, 1] - y_i)**2)
        
        # Compute the local Lipschitz underestimate for this sample
        Z_local_under = f_x_i_y_i - L * distances
        
        # Update the global underestimate by taking the maximum across samples
        Z_under = np.maximum(Z_under, Z_local_under)
    
    return Z_under

def estimate_lipschitz_constant(samplesXY, f_values):

    """
    Estimate the Lipschitz constant based on known sample points and their function values.
    
    Parameters:
    samplesXY: np.ndarray of shape (n_samples, 2)
        Array of the sampled (x, y) points.
    f_values: np.ndarray of shape (n_samples,)
        Array of function values at the sampled points.
    
    Returns:
    L_est: float
        The estimated Lipschitz constant.
    """
    n_samples = samplesXY.shape[0]
    # print(samplesXY.shape)
    # print(n_samples)
    L_est = 0.0
    
    # Loop over all pairs of points to estimate L
    for i in range(n_samples):
        for j in range(i + 1, n_samples):
            # Compute the Euclidean distance between points i and j
            dist = np.linalg.norm(samplesXY[i] - samplesXY[j])
            if dist > 0:
                # Compute the absolute difference in function values
                f_diff = np.abs(f_values[i] - f_values[j])
                # Compute the slope and update the maximum
                # print(L_est, f_diff/dist)
                L_est = max(L_est, f_diff / dist)
    
    return L_est

class RBFSurrogateModel:
    def __init__(self, epsilon=1.0):
        """
        Initialize the RBF Surrogate Model.

        Parameters:
        epsilon: float
            The shape parameter for the Gaussian RBF kernel. It controls the width of the Gaussian basis function.
        """
        self.epsilon = epsilon
        self.centers = None
        self.weights = None

    def _rbf_kernel(self, X1, X2):
        """
        Compute the RBF (Gaussian) kernel between two sets of data points.

        Parameters:
        X1: ndarray of shape (n_samples_X1, n_features)
            First set of data points.
        X2: ndarray of shape (n_samples_X2, n_features)
            Second set of data points.
        
        Returns:
        K: ndarray of shape (n_samples_X1, n_samples_X2)
            Kernel matrix.
        """
        # Compute the pairwise Euclidean distance between points in X1 and X2
        distances = cdist(X1, X2, 'euclidean')
        # Gaussian RBF: exp(-epsilon^2 * distance^2)
        K = np.exp(-(self.epsilon * distances) ** 2)
        return K

    def fit(self, X_train, y_train):
        """
        Train the RBF surrogate model on the training data.

        Parameters:
        X_train: ndarray of shape (n_samples, n_features)
            Training data inputs.
        y_train: ndarray of shape (n_samples,)
            Training data targets.
        """
        # print('xtrain', X_train.shape)
        self.centers = X_train  # The centers of the RBF are the training data points
        # Compute the RBF kernel matrix for the training data
        K_train = self._rbf_kernel(X_train, X_train)
        # Solve the linear system K * weights = y_train to get the weights
        self.weights = np.linalg.solve(K_train, y_train)

    def predict(self, X_test):
        """
        Make predictions for new input data using the trained model.

        Parameters:
        X_test: ndarray of shape (n_samples_test, n_features)
            New data inputs for which predictions are to be made.

        Returns:
        y_pred: ndarray of shape (n_samples_test,)
            Predicted values for the input data.
        """
        # Compute the RBF kernel between the test data and the centers
        K_test = self._rbf_kernel(X_test, self.centers)
        # Multiply the kernel matrix by the weights to get predictions
        #apparently @ is python syntax for matrix multiplication...
        y_pred = K_test @ self.weights
        return y_pred
    
def objective_function(vec):
    """Objective function wrapper for optimization.
    Args:
        vec (np.ndarray): A vector representing candidate solution (x, y).
    Returns:
        float: Fitness value of the solution.
    """
    x, y = vec
    return ackley_function(x, y)

# Differential Evolution Optimizer
class LSADE:
    def __init__(self, bounds, pop_size=50, mutation_factor=0.8, crossover_prob=0.7, method='random'):
        """
        Initialize the Differential Evolution (DE) optimizer.
        
        Parameters:
        bounds (list of tuple): List of (min, max) bounds for each dimension.
        pop_size (int): Number of candidate solutions in the population.
        mutation_factor (float): Scaling factor for mutation [0, 2].
        crossover_prob (float): Crossover probability [0, 1].
        max_generations (int): Maximum number of generations to evolve.
        method (str): Population initialization method ('random' or 'lhs').
        """
        self.bounds = np.array(bounds)
        self.dimensions = len(bounds)
        self.pop_size = pop_size
        self.mutation_factor = mutation_factor
        self.crossover_prob = crossover_prob
        # self.max_generations = max_generations
        self.method = method
        self.feFeatures = np.empty((0,2))
        self.feTargets = np.empty(0)        
        # Initialize population
        self.population = self.initialize_population()

        self.best_solution = None
        self.best_fitness = np.inf
    
    def initialize_population(self):
        """Initialize population using random sampling or Latin Hypercube Sampling."""

        # print('initial shape', self.feFeatures.shape)
        if self.method == 'lhs':
            # Latin Hypercube Sampling
            sampler = qmc.LatinHypercube(d=self.dimensions)
            sample = sampler.random(n=self.pop_size)
            population = qmc.scale(sample, self.bounds[:, 0], self.bounds[:, 1])
        else:
            # Random Sampling
            population = np.random.rand(self.pop_size, self.dimensions)
            for i in range(self.dimensions):
                population[:, i] = self.bounds[i, 0] + population[:, i] * (self.bounds[i, 1] - self.bounds[i, 0])

        for i in range(0, len(population)):
            self.feTargets = np.append(self.feTargets, objective_function(population[i]))
            self.feFeatures = np.vstack((self.feFeatures, population[i]))
        
        # print('final shape', self.feFeatures.shape)

        return population
    
    def mutate(self, target_idx):
        """Mutation using DE/best/1 strategy."""
        # Choose three random and distinct individuals different from target_idx
        indices = [idx for idx in range(self.pop_size) if idx != target_idx]
        np.random.shuffle(indices)
        r1, r2 , r3= indices[:3]
        
        # Best individual in current population
        best_idx = np.argmin([objective_function(ind) for ind in self.population])
        best = self.population[best_idx]
        
        # Mutant vector: v = best + F * (r1 - r2)
        mutant = best + self.mutation_factor * (self.population[r1] - self.population[r2])
        
        # Ensure mutant vector is within bounds
        mutant = np.clip(mutant, self.bounds[:, 0], self.bounds[:, 1])
        
        return mutant
    
    def crossover(self, target, mutant):
        """Crossover to create trial vector.
        This loops through the features in the vector (dimensions) and sees if any of them crossover. 
        allows the retention of some original vector features but not others. 
        """

        trial = np.copy(target)
        # print(trial.shape)
        # print(mutant.shape)
        for i in range(self.dimensions):
            if np.random.rand() < self.crossover_prob or i == np.random.randint(self.dimensions):
                # print(trial[i], mutant[i])

                trial[i] = mutant[i]
        return trial
            

    # def select(self, target, trial):
    #     """Selection: Return the individual with the better fitness."""
    #     if objective_function(trial) < objective_function(target):
    #         return trial
    #     return target
    
    def localRBF(self, numSolutions):

        bestFeatures = np.empty((numSolutions,2))
        bestTargets = np.empty(numSolutions)

        #find c best solutions
        bestIndices = np.argsort(self.feTargets)[:numSolutions]

        for i in range(numSolutions):
            bestFeatures[i] = self.feFeatures[bestIndices[i]]
            bestTargets[i] = self.feTargets[bestIndices[i]]




        x_min, x_max = np.min(bestFeatures[:, 0]), np.max(bestFeatures[:, 0])
        y_min, y_max = np.min(bestFeatures[:, 1]), np.max(bestFeatures[:, 1])

        bounds = [(x_min, x_max), (y_min, y_max)]

        # pairwiseDistancesLocal = np.linalg.norm(bestFeatures[:, np.newaxis] - bestFeatures, axis=2)
        # avgDistanceLocal = np.mean(pairwiseDistancesLocal)

        localGP = GPTrain(bestFeatures, bestTargets, meanPrior='max')

        # localRBF = RBFSurrogateModel(epsilon=1.0)
        # localRBF.fit(bestFeatures, bestTargets)

        # functionEval = localRBF.predict()
        localDE = DifferentialEvolution(bounds, localGP )
        bestLocalSolution, bestLocalFitness = localDE.optimize()

        return bestLocalSolution
    
    def optimizerStep(self):
        """Run the Differential Evolution optimization."""
        # x_range = np.linspace(-5, 5, 100)
        # y_range = np.linspace(-5, 5, 100)
        # X, Y = np.meshgrid(x_range, y_range)
        # Z = ackley_function(X, Y)

        # for generation in range(self.max_generations):
        new_population = np.zeros_like(self.population)
            
        for i in range(self.pop_size):
            # print(i)
            target = self.population[i]
            mutant = self.mutate(i)
            trial = self.crossover(target, mutant)
            #do not evaluate target/trial vectors, instead just save the trial vectors as the new population
            #so they can be evaluated on the global RBF and Lipschitz surrogates later
            # new_population[i] = self.select(target, trial)
            new_population[i] = trial
            
            # Update the population
        self.population = new_population
        # print(new_population.shape)
        # pairwise_distances = np.linalg.norm(self.feFeatures[:,np.newaxis] - self.feFeatures, axis=2)
        # avg_distance = np.mean(pairwise_distances)
        # globalRBF = RBFSurrogateModel(epsilon=1.0)

        # globalRBF.fit(self.feFeatures, self.feTargets)

        GPModel = GPTrain(self.feFeatures, self.feTargets, meanPrior='zero')

        popOnGP = GPEval(GPModel, self.population)

        
        #evaluating whole landscape on RBF for plotting reasons:
        x_range = np.linspace(-5,5,50)
        y_range = np.linspace(-5,5,50)
        fullRange = list(product(x_range, y_range))
        fullRangeArray = np.array(fullRange)
        y_pred = GPEval(GPModel, fullRangeArray)


        #evaluate current population (children) on RBF
        # popOnRBF = globalRBF.predict(self.population)

        #function evaluation of best predicted child
        best_idx = np.argmin(popOnGP)

        bestFeature = self.population[best_idx]

        # print('best index =', best_idx)
        #evaluate best child and add results to global stores of FE features and targets
        self.feTargets = np.append(self.feTargets, objective_function(self.population[best_idx]))
        self.feFeatures = np.vstack((self.feFeatures, self.population[best_idx]))

        plt.scatter(fullRangeArray[:,0], fullRangeArray[:,1], c = y_pred)

        plt.scatter(self.population[:, 0], self.population[:, 1], color='red', label='Final Population', s=5)
        plt.scatter(bestFeature[0], bestFeature[1], color='blue', label='Best Solution', s=10)
        # plt.legend()
        plt.title("Global Surrogate")
        plt.colorbar()
        plt.clim(0,14)
        plt.savefig('globalGP.png')
        plt.close()
        #generate Lipschitz surrogate, evaluate children, FE evaluate best potential child and add to bank

        # print(self.feTargets, self.feFeatures)
        # print('final shape', self.feFeatures.shape)

        L_est = estimate_lipschitz_constant(self.feFeatures, self.feTargets)

        popOnLipschitz = lipschitz_global_underestimate(self.feTargets, self.feFeatures, L_est, self.population)
        best_idx = np.argmin(popOnLipschitz)
        bestFeature = self.population[best_idx]

        self.feTargets = np.append(self.feTargets, objective_function(self.population[best_idx]))
        self.feFeatures = np.vstack((self.feFeatures, self.population[best_idx]))        

        #evaluating all points in function on Lipschitz for plotting purposes
        Z_under = lipschitz_global_underestimate(self.feTargets, self.feFeatures, L_est, fullRangeArray)

        plt.scatter(fullRangeArray[:,0], fullRangeArray[:,1], c = Z_under)

        plt.scatter(self.population[:, 0], self.population[:, 1], color='red', label='Final Population', s=5)
        plt.scatter(bestFeature[0], bestFeature[1], color='blue', label='Best Solution', s=10)
        # plt.legend()
        plt.title("Lipschitz Underestimation")
        plt.colorbar()
        plt.clim(0,14)
        plt.savefig('lipschitz.png')
        plt.close()



        #construct local RBF using c best solutions, find minima using DE, and evaluate at that minima

        bestLocalSolution = self.localRBF(15)

        bestLocalSolution = np.reshape(bestLocalSolution, (2,))

        print('best local Solution', bestLocalSolution)

        self.feTargets = np.append(self.feTargets, objective_function(bestLocalSolution))
        self.feFeatures = np.vstack((self.feFeatures, bestLocalSolution)) 

        plt.scatter(self.feFeatures[:,0], self.feFeatures[:,1], c = self.feTargets)
        plt.title('Evaluated Population')
        plt.colorbar()
        plt.clim(0,14)
        plt.savefig('population.png')
        plt.close()

        # Track the best solution
        # best_idx = np.argmin([objective_function(ind) for ind in self.population])
        # best_fitness = objective_function(self.population[best_idx])
        
        # if best_fitness < self.best_fitness:
        #     self.best_fitness = best_fitness
        #     self.best_solution = self.population[best_idx]
            
            # plt.contourf(X, Y, Z, levels=50, cmap='viridis')
            # plt.scatter(de.population[:, 0], de.population[:, 1], color='red', label='Final Population', s=5)
            # # plt.scatter(best_solution[0], best_solution[1], color='blue', label='Best Solution', s=100)
            # plt.legend()
            # plt.title("Ackley Function with Final Population and Best Solution")
            # plt.colorbar()
            # plt.show()
            # Debug information
            # print(f"Generation {generation + 1}: Best Fitness = {self.best_fitness}")
        print('Best found solution = ', min(self.feTargets))


        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")

        localGP = Image.open('localGP.png')
        globalGP = Image.open('globalGP.png')
        lipschitz = Image.open('lipschitz.png')
        population = Image.open('population.png')

        width, height = localGP.size
        combinedImage = Image.new('RGB', (2 * width, 2 * height))
        combinedImage.paste(localGP, (0, 0))
        combinedImage.paste(lipschitz, (width, 0))
        combinedImage.paste(globalGP, (0, height))
        combinedImage.paste(population, (width, height))

        combinedImage.save(f'plots/{timestamp}.png')


        return self.best_solution, self.best_fitness
    

        

    

    


In [76]:
bounds = [(-5,5), (-5,5)] #bounds for each dimension (x and y)

#initialise DE solver
LSADE = LSADE(bounds=bounds, pop_size=50, method='lhs')

iter = 0

while iter < 50:
    LSADE.optimizerStep()
    print(LSADE.population.shape)
    iter += 1



/var/folders/8p/wlgb17ln00dg_lln7zq_3gpw0000gn/T/ipykernel_3307/4061782729.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.mean_module.constant.data = torch.tensor(torch.max(train_y))


Generation 200: Best RBF Fitness = [4.09392402]
best local Solution [-0.00792783  0.3012966 ]
Best found solution =  2.146502457252567
(50, 2)
Generation 200: Best RBF Fitness = [1.97587113]
best local Solution [-0.16703728  0.22014475]
Best found solution =  2.076729373844906
(50, 2)
Generation 200: Best RBF Fitness = [1.08836919]
best local Solution [-0.02088193  0.09772967]
Best found solution =  0.5285992056711
(50, 2)
Generation 200: Best RBF Fitness = [-0.33731822]
best local Solution [ 0.13363946 -0.07007954]
Best found solution =  0.38439515728279305
(50, 2)
Generation 200: Best RBF Fitness = [0.2366061]
best local Solution [0.00379194 0.00731433]
Best found solution =  0.025109510466287066
(50, 2)
Generation 200: Best RBF Fitness = [-0.00845021]
best local Solution [ 0.00629551 -0.02226622]
Best found solution =  0.025109510466287066
(50, 2)
Generation 200: Best RBF Fitness = [0.0269633]
best local Solution [0.01504517 0.00518854]
Best found solution =  0.025109510466287066
(5

In [77]:
import glob
import imageio

image_files = sorted(glob.glob("plots/*.png"))
# print('here')
images = [Image.open(img) for img in image_files]

# Step 3: Save images as a GIF
if images:
    # 'duration' is the time each frame stays on screen (milliseconds)
    images[0].save(
        "output6.gif",
        save_all=True,
        append_images=images[1:],  # Include the rest of the images
        duration=400,               # Duration of each frame in milliseconds
        loop=0                       # 0 means loop indefinitely
    )